# PGVectorStore

`PGVectorStore` is an implementation of a LangChain vectorstore using `postgres` as the backend.

This notebook goes over how to use the `PGVectorStore` API.

The code lives in an integration package called: [langchain-postgres](https://github.com/langchain-ai/langchain-postgres/).

## Setup

This package requires a PostgreSQL database with the `pgvector` extension.

You can run the following command to spin up a container for a `pgvector` enabled Postgres instance:

```shell
docker run --name pgvector-container -e POSTGRES_USER=langchain -e POSTGRES_PASSWORD=langchain -e POSTGRES_DB=langchain -p 6024:5432 -d pgvector/pgvector:pg16
```

### Install
Install the integration library, `langchain-postgres`.

In [ ]:
%pip install --upgrade --quiet  langchain-postgres
# This notebook also requires the following dependencies
%pip install --upgrade --quiet  langchain-core langchain-cohere sqlalchemy

### Set your Postgres values

Set your Postgres values to test the functionality in this notebook against a Postgres instance.

In [2]:
# @title Set your values or use the defaults to connect to Docker { display-mode: "form" }
POSTGRES_USER = "langchain"  # @param {type: "string"}
POSTGRES_PASSWORD = "langchain"  # @param {type: "string"}
POSTGRES_HOST = "localhost"  # @param {type: "string"}
POSTGRES_PORT = "6024"  # @param {type: "string"}
POSTGRES_DB = "langchain"  # @param {type: "string"}
TABLE_NAME = "vectorstore"  # @param {type: "string"}
VECTOR_SIZE = 1024  # @param {type: "int"}

## Initialization

### PGEngine Connection Pool

One of the requirements and arguments to establish PostgreSQL as a vector store is a `PGEngine` object. The `PGEngine`  configures a shared connection pool  to your Postgres database. This is an industry best practice to manage number of connections and to reduce latency through cached database connections.

`PGVectorStore` can be used with the `asyncpg` and `psycopg3` drivers.

To create a `PGEngine` using `PGEngine.from_connection_string()` you need to provide:

1. `url` : Connection string using the `postgresql+asyncpg` driver.


**Note:** This tutorial demonstrates the async interface. All async methods have corresponding sync methods.

In [3]:
# See docker command above to launch a Postgres instance with pgvector enabled.
CONNECTION_STRING = (
    f"postgresql+asyncpg://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}"
    f":{POSTGRES_PORT}/{POSTGRES_DB}"
)
# To use psycopg3 driver, set your connection string to `postgresql+psycopg://`

In [4]:
from langchain_postgres import PGEngine

pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)

To create a `PGEngine` using `PGEngine.from_engine()` you need to provide:

1. `engine` : An object of `AsyncEngine`

In [5]:
from sqlalchemy.ext.asyncio import create_async_engine

# Create an SQLAlchemy Async Engine
engine = create_async_engine(
    CONNECTION_STRING,
)

pg_engine = PGEngine.from_engine(engine=engine)

### Initialize a table

The `PGVectorStore` class requires a database table. The `PGEngine` engine has a helper method `ainit_vectorstore_table()` that can be used to create a table with the proper schema for you.

See [Create a custom Vector Store](#create-a-custom-vector-store) or [Create a Vector Store using existing table](#create-a-vector-store-using-existing-table) for customizing the schema.

In [7]:
await pg_engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=VECTOR_SIZE,
)

#### Optional Tip: 💡
You can also specify a schema name by passing `schema_name` wherever you pass `table_name`. Eg:

```python
SCHEMA_NAME="my_schema"

await pg_engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=768,
    schema_name=SCHEMA_NAME,    # Default: "public"
)
```

### Create an embedding class instance

You can use any [LangChain embeddings model](https://python.langchain.com/docs/integrations/text_embedding/).

In [8]:
from langchain_cohere import CohereEmbeddings

embedding = CohereEmbeddings(model="embed-english-v3.0")

### Initialize a default PGVectorStore

Use the default table schema to connect to the vectorstore.

See [Create a custom Vector Store](#create-a-custom-vector-store) or [Create a Vector Store using existing table](#create-a-vector-store-using-existing-table) for customizing the schema.

In [9]:
from langchain_postgres import PGVectorStore

store = await PGVectorStore.create(
    engine=pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=embedding,
)

## Manage vector store

### Add documents

Add documents to the vector store. Metadata is stored in a JSON column, see "Create a custom Vector Store" to store metadata to be used for filters.

In [ ]:
import uuid

from langchain_core.documents import Document

docs = [
    Document(
        id=str(uuid.uuid4()),
        page_content="Red Apple",
        metadata={"description": "red", "content": "1", "category": "fruit"},
    ),
    Document(
        id=str(uuid.uuid4()),
        page_content="Banana Cavendish",
        metadata={"description": "yellow", "content": "2", "category": "fruit"},
    ),
    Document(
        id=str(uuid.uuid4()),
        page_content="Orange Navel",
        metadata={"description": "orange", "content": "3", "category": "fruit"},
    ),
]

await store.aadd_documents(docs)

### Add texts

Add text directly to the vectorstore, if not structured as a Document.

In [ ]:
import uuid

all_texts = ["Apples and oranges", "Cars and airplanes", "Pineapple", "Train", "Banana"]
metadatas = [{"len": len(t)} for t in all_texts]
ids = [str(uuid.uuid4()) for _ in all_texts]

await store.aadd_texts(all_texts, metadatas=metadatas, ids=ids)

### Delete documents

Documents can be deleted using ids.

In [ ]:
await store.adelete([ids[1]])

## Query vector store

### Search for documents

Use a natural language query to search for similar documents.

In [ ]:
query = "I'd like a fruit."
docs = await store.asimilarity_search(query)
print(docs)

### Search for documents by vector

Search for similar documents using a vector embedding.

In [ ]:
query_vector = embedding.embed_query(query)
docs = await store.asimilarity_search_by_vector(query_vector, k=2)
print(docs)

## Add a Index
Speed up vector search queries by applying a vector index. Learn more about [vector indexes](https://cloud.google.com/blog/products/databases/faster-similarity-search-performance-with-pgvector-indexes). 

Indexes will use a default index name if a name is not provided. To add multiple indexes, different index names are required.

In [ ]:
from langchain_postgres.v2.indexes import HNSWIndex, IVFFlatIndex

index = IVFFlatIndex()
await store.aapply_vector_index(index)

index = HNSWIndex(name="my-hnsw-index")
await store.aapply_vector_index(index)

Set index parameters to tune the index for optimal balance between recall and QPS.

In [ ]:
index = IVFFlatIndex(name="my-ivfflat", lists=120)
await store.aapply_vector_index(index)

### Re-index

Rebuild an index using the data stored in the index's table, replacing the old copy of the index. Some index types may require re-indexing after a considerable amount of new data is added.

In [ ]:
await store.areindex()  # Re-index using the default index name
await store.areindex("my-hnsw-index")  # Re-index using the index name

### Drop an index

Remove a vector index.

In [ ]:
await store.adrop_vector_index()  # Delete index using the default name
await store.adrop_vector_index("my-hnsw-index")  # Delete index using the index name

## Create a custom Vector Store

Customize the vectorstore with special column names or with custom metadata columns.

`ainit_vectorstore_table`
* Use fields `content_column`, `embedding_column`,`metadata_columns`, `metadata_json_column`, `id_column` to rename the columns. 
* Use the `Column` class to create custom id or metadata columns. A Column is defined by a name and data type. Any Postgres [data type](https://www.postgresql.org/docs/current/datatype.html) can be used.
* Use `store_metadata` to create a JSON column to store extra metadata.

#### Optional Tip: 💡
To use non-uuid ids, you must customize the id column:
```python
await pg_engine.ainit_vectorstore_table(
    ...,
    id_column=Column(name="langchain_id", data_type="INTEGER")
)
```

`PGVectorStore`
* Use fields `content_column`, `embedding_column`,`metadata_columns`, `metadata_json_column`, `id_column` to rename the columns. 
* `ignore_metadata_columns` to ignore columns that should not be used for Document metadata. This is helpful when using a preexisting table, where all data columns are not necessary.
* Use a different `distance_strategy` for the similarity calculation during vector search.
* Use `index_query_options` to tune local index parameters during vector search.

In [19]:
from langchain_postgres import Column

# Set table name
TABLE_NAME = "vectorstore_custom"
# SCHEMA_NAME = "my_schema"

await pg_engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    vector_size=VECTOR_SIZE,
    metadata_columns=[Column("len", "INTEGER")],
)


# Initialize PGVectorStore
custom_store = await PGVectorStore.create(
    engine=pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=embedding,
    metadata_columns=["len"],
)

### Search for documents with metadata filter

A Vector Store can take advantage of relational data to filter similarity searches. The vectorstore supports a set of filters that can be applied against the metadata fields of the documents. See the [migration guide](https://github.com/langchain-ai/langchain-postgres/blob/main/examples/migrate_pgvector_to_pgvectorstore.ipynb) for details on how to migrate to use metadata columns.

`PGVectorStore` currently supports the following operators and all Postgres data types.

| Operator  | Meaning/Category        |
|-----------|-------------------------|
| $eq       | Equality (==)           |
| $ne       | Inequality (!=)         |
| $lt       | Less than (\<)           |
| $lte      | Less than or equal (\<=) |
| $gt       | Greater than (>)        |
| $gte      | Greater than or equal (>=) |
| $in       | Special Cased (in)      |
| $nin      | Special Cased (not in)  |
| $between  | Special Cased (between) |
| $exists   | Special Cased (is null) |
| $like     | Text (like)             |
| $ilike    | Text (case-insensitive like) |
| $and      | Logical (and)           |
| $or       | Logical (or)            |

In [ ]:
import uuid

docs = [
    Document(
        id=str(uuid.uuid4()),
        page_content="Red Apple",
        metadata={"description": "red", "content": "1", "category": "fruit"},
    ),
    Document(
        id=str(uuid.uuid4()),
        page_content="Banana Cavendish",
        metadata={"description": "yellow", "content": "2", "category": "fruit"},
    ),
    Document(
        id=str(uuid.uuid4()),
        page_content="Orange Navel",
        metadata={"description": "orange", "content": "3", "category": "fruit"},
    ),
]

await custom_store.aadd_documents(docs)

# Use a dictionary filter on search
docs = await custom_store.asimilarity_search(query, filter={"content": {"$gte": 1}})

print(docs)

## Create a Vector Store using existing table

A Vector Store can be built up on an existing table.

Assuming there's a pre-existing table in PG DB: `products`, which stores product details for an eComm venture.

<details>
  <summary>Click for Table Schema Details</summary>

  ### SQL query for table creation
  ```
    CREATE TABLE products (
        product_id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        description TEXT,
        price_usd DECIMAL(10, 2) NOT NULL,
        category VARCHAR(255),
        quantity INT DEFAULT 0,
        sku VARCHAR(255) UNIQUE NOT NULL,
        image_url VARCHAR(255),
        metadata JSON,
        embed vector(768) DEFAULT NULL -- vector dimensions depends on the embedding model
    );
  ```
  ### Insertion of records
  ```
INSERT INTO
  products (name,
    description,
    price_usd,
    category,
    quantity,
    sku,
    image_url,
    METADATA,
    embed)
VALUES
  ('Laptop', 'High-performance gaming laptop', 1200.00, 'Electronics', 10, 'SKU12345', 'https://example.com/laptop.jpg', '{"category" : "Electronics", "name" : "Laptop", "description" : "High-performance gaming laptop"}', ARRAY[0.028855365,-0.012488421,0.006031946,0.0041402685,0.058347773,0.034766156,0.0033533745,0.02021188,0.022670388,0.049201276,0.029006215,-0.00986186,-0.052214462,-0.012280585,0.023684537,-0.059519604,0.001378169,-0.04670758,0.020753963,0.0013795564,0.013659675,0.013842887,-0.011299884,-0.03746782,-0.024693582,-0.07013125,0.030126512,-0.028513059,-0.045777187,0.020505989,-0.05952914,0.0015648323,-0.050879195,0.006477519,-0.007886009,-0.02629686,-0.0161126,0.0314275,-0.0328995,0.0265609,-0.01530363,-0.019561788,-0.04535006,0.030131247,0.05462397,-0.0122205755,0.009777537,-0.0049046725,0.02023674,-0.064513534,0.041379478,0.006994005,0.045187026,-0.029661352,0.019398877,-0.02221874,-0.017291287,-0.016321573,-0.033429787,-0.009547383,0.031690586,0.009064364,-0.015285908,0.076494075,0.010917006,-0.016593782,-0.018348552,0.017040739,0.05943369,-0.020822933,0.009285482,0.027736548,0.07029796,-0.0644397,-0.037717465,-0.047550958,-0.0054535423,0.047678974,0.060069297,-0.015072207,-0.04320405,-0.0019738402,-0.061910342,-0.034316592,-0.023359261,0.057676528,0.0054635284,0.042063717,0.020484874,0.005591504,-0.008757174,-0.0153757995,0.04932489,-0.04626516,0.0004756786,0.03749645,0.018522505,-0.015642159,-0.00842546,-0.06284679,-0.006150201,-0.061204597,0.0008340049,0.0040505463,0.014210282,-0.009027461,-0.014203488,0.030791085,-0.022282222,0.0011378798,-0.047313087,-0.008226634,-0.03726029,-0.04307269,0.04519085,-0.021895533,0.019570287,0.08584432,-0.003815025,0.021276724,0.027253378,-0.01660856,0.056772888,0.053538952,0.02739156,0.04655151,0.021516826,0.064367436,-0.021094408,0.0149244,-0.009901731,-0.04166729,-0.0032499651,0.022982895,0.063407354,0.04826923,0.056767307,-0.024418632,0.063300684,0.08071309,-0.054988176,0.01652395,-0.014671885,0.000837919,-0.044569198,0.03651631,-0.016364796,0.0053244857,0.051150765,-0.01878448,0.005112729,-0.0011729974,-0.052268386,0.034706745,0.05072015,0.0052968785,0.021704907,0.045661792,0.002976117,-0.02205154,0.037168674,0.002627892,0.018275578,0.032312263,-0.06719407,-0.056915596,-0.019727554,0.0009450171,0.0029568567,0.047435578,0.033826437,-0.009351167,-0.05718618,-0.062166944,-0.005684254,-0.009788955,0.016364967,0.0122847315,-0.016126394,0.012999976,-0.075272575,0.017478324,0.03005914,0.024401167,0.0099941185,-0.043311242,0.032115143,0.0047207233,-0.034337096,0.0054743756,-0.0024234303,0.012045114,0.032277416,-0.019994166,0.012312445,0.021211047,-0.037350595,0.0017910452,0.04450775,0.0054527316,0.03591427,0.029365221,0.0009824947,-0.006488191,0.034008037,0.01649739,0.07955305,-0.035204325,0.0056851353,-0.0086927805,-0.032573096,0.0010878195,-0.061459325,0.027879931,0.015068312,0.032717325,0.03890655,0.01902891,0.016527452,-0.0020142202,0.025338948,-0.0016015576,-0.06429177,-0.0041105347,-0.025726322,0.09078289,-0.03174613,0.015951345,0.009411334,-0.03598392,0.034463316,0.010011217,-0.009883364,-0.008042991,0.040896636,-0.025115138,0.048056312,0.028382989,0.007793395,0.019581616,-0.02584373,0.04317992,0.025689745,0.02035658,-0.05990108,-0.0007803719,-0.06793038,-0.02130707,0.0048890263,0.042799927,-0.009928141,-0.003192067,0.008781545,0.024785394,-0.07565836,-0.043356933,-0.067785084,-0.019649943,-0.024896448,-0.008327102,-0.015189734,-0.0140810255,0.0049958434,-0.015353841,0.020730853,0.028829988,-0.022614283,-0.03751693,0.011577282,0.031927988,-0.024855413,-0.042680055,0.08018929,-0.0021632465,-0.017928878,-0.0030442774,-0.005651566,-0.0010570051,-0.040446285,-0.00189408,0.06388222,0.0024985478,0.004886204,-0.05113467,-0.019480383,0.049765434,0.0077566532,-0.07356923,0.011988718,-0.020965552,-0.04025921,-0.032686763,-0.0053743063,-0.015599607,-0.03576176,0.00907552,-0.044702522,0.038329247,0.046024352,0.02194124,0.01844749,0.004619246,-0.029577129,-0.031205669,0.00896738,0.0115034515,0.013058729,0.01372364,0.03063813,-0.0316296,-0.04826321,-0.049244087,-0.037644744,0.019473651,0.059536345,0.04033204,-0.06602803,0.050612085,-0.027031716,0.04213856,-0.015262794,0.07257449,0.044631373,-0.0151061565,0.012033797,0.0009732858,-0.014827035,0.046652585,-0.042083394,-0.0436095,-0.035586536,0.026696088,-0.004066648,0.06954644,-0.029623765,-0.020358749,-0.04957031,0.01740737,-0.017026579,0.011162373,0.0487351,-0.031720005,-0.050231773,-0.089686565,-0.014156863,-0.02636994,0.015916161,-0.025308851,0.02081637,-0.02257452,0.021604244,0.10139386,-0.03208752,-0.008580313,-0.008898747,-0.06853021,0.04102758,0.041922912,0.047566738,-0.0341902,-0.07725792,0.005653997,0.00021225312,-0.0104829185,0.001749244,0.011929626,0.078264005,0.036519475,-0.0073295147,0.021337496,-0.008336836,-0.035804152,0.010720447,0.007127837,-0.053885818,-0.009795316,-0.05424524,-0.003111704,0.019710006,-0.012413589,0.02320744,0.024137065,0.023079542,0.0030920266,0.013961592,0.0040291087,0.020265838,0.041183334,-0.0029272675,-0.018539282,-0.011489972,0.017938145,0.025854694,0.033188265,-0.042004097,-0.0106819095,-0.045249976,-0.06986475,0.030204961,-0.032193515,-0.00095170306,-0.0107111735,-0.017970158,-0.02740307,-0.06307846,-0.031544626,0.004178074,0.016592229,-0.032037992,-0.030618787,0.008946463,0.03110429,0.0207187,-0.016861247,-0.08070464,-0.03067543,0.067448415,-0.041909873,-0.0048193526,0.018761802,0.020243261,0.024184326,0.002299031,-0.014152546,-0.035749547,-0.0071563246,0.050069712,-0.027215304,0.049641047,0.02778935,0.070745096,0.023794815,0.0029510225,0.0069351746,-0.034430653,-0.085317925,-0.036851004,0.023848707,0.035138704,-0.017030267,0.041982725,0.014077844,0.012787886,-0.029716792,-0.024732213,-0.059604853,0.024058796,-0.027469097,0.02969232,-0.06889772,-0.034953564,-0.0678685,0.02039748,-0.073483475,-0.04067064,-0.023628144,0.052601792,0.10005532,0.0027910264,-0.00044562414,0.025615653,0.008896907,-0.016369712,-0.030180404,0.026393086,-0.02041892,0.0072918,-0.018448602,0.020845268,0.006290655,-0.010850651,-0.035378493,-0.01083432,0.012116494,-0.045438327,0.05191333,-0.082797736,0.042320468,0.039703712,0.00923727,0.03598509,-0.064069025,0.049349498,0.007205401,-0.0079013845,0.015407162,-0.049755134,-0.0335355,-0.033252683,0.025886077,-0.043650113,-0.021745201,-0.046847582,-0.02873071,-0.01435186,0.01642749,-0.030346846,0.00564007,0.0074587157,0.027222605,-0.024691164,0.007528186,-0.04551536,-0.011026097,0.091698915,-0.062147886,0.0013525741,-0.0065618614,-0.030818032,0.024246406,-0.010786434,0.006758053,-0.016815495,0.071824,0.022536254,-0.026362726,-0.066206455,0.011966612,0.06430261,0.021586932,0.032340884,-0.015460002,-0.0963993,-0.0041012894,0.026189657,-0.101343565,0.038662393,0.07043264,-0.0373032,0.0038455573,-0.017408002,0.12948644,-0.056175977,0.02693295,-0.033682294,-0.032874268,0.0016187532,0.023056049,0.06884863,0.04350595,0.02135146,-0.059129357,-0.0055416543,0.0098204445,-0.008596177,-0.04332969,-0.012624592,-0.09298762,0.041691724,-0.014171953,0.004045705,0.009756654,0.059401184,-0.02852561,0.006892971,-0.019445946,-0.013781522,-0.03458903,-0.001079532,-0.008455719,-0.025446072,-0.03641567,-0.034449898,0.004487285,0.07899037,0.031314176,-0.031828023,0.031026838,0.034468375,0.0166286,0.032397788,0.02265452,0.07575427,0.015329588,0.05969185,-0.049144097,-0.043501142,0.031721197,-0.03434621,0.04558533,-0.00039121095,0.00093291467,0.033810064,0.0131731015,-0.0161992,0.039637238,0.0018543458,-0.041811496,-0.01406263,-0.020126836,-0.011859638,0.029031854,0.018889664,0.015262868,-0.03756649,-0.024570176,0.02538295,0.0038968727,-0.06393701,0.00093783275,-0.05943941,-0.062095385,0.08169533,-0.026443593,0.045758378,-0.026765708,-0.023990292,-0.028646782,0.0013627055,0.0022589415,0.009424216,-0.004252787,0.01159273,-0.0393901,-0.02593045,-0.04785985,0.023880653,0.012857186,-0.028907716,-0.05117687,-0.017512657,-0.035777926,0.01183514,0.025101895,0.089760125,-0.009716518,0.012040118,-0.023447596,0.057904292,0.03486462,-0.014875794,0.05191007,0.002385196,0.016686346,-0.052348964,-0.029286617,0.023832947,-0.02915365,0.007727999,-0.012708917,-0.055755604,-0.0073897606,0.032306697,0.02891973,-0.029123511,0.08987496,0.049180396,-0.08122004,-0.029804248,0.03262262,-0.06680825,0.016717656,0.0038353673,0.021287518,0.0018424556,-0.0041867862,-0.0011719886,-0.044280436,0.02019424,-0.052992586,-0.05063449,0.039644204,-0.0494374,-0.033791043,-0.0041454337,-0.032513123,-0.073564336,-0.04585872,0.0023792102,0.027335508,-0.06999816,0.04888005,0.026423248,0.021874929,0.010904174,0.060097646,-0.034017522,0.05548881,-0.024519302,0.049890403,-0.015645353,-0.060680103,0.017045638,0.019808227,0.025153033,0.0040058065,0.053807795,0.034485374,-0.053428553,-0.0034872151,0.033813756,-0.03047597,0.007858348,0.024711734,0.060215656,0.008143574,-0.0070263194,0.0048007956,0.015641727,0.052094024,-0.049206913,0.016296484,-0.0059813466,0.040864628,0.013278136,-0.012139221,-0.04106141,0.0144868875,0.0013842004,0.021345256,0.04826021,-0.06929805,-0.021199407,0.00090551435,0.009481861,-0.0017141728,0.028452767,-0.019797614,0.038415838,0.056153923,-0.014074272,-0.00823969,-0.00050664565,-0.07698735,-0.025168924,0.057516575,-0.07501726,0.037316702,-0.02765656,-0.011325112,0.058868058,-0.010426108,-0.013318932,-0.0016809561,-0.062076304,0.027063645,-0.020674324,0.06843111,0.018448142,-0.04226709,-0.015164476,-0.008888517,0.040828817,0.048462827,0.00942803,-0.019631634,0.020950766,-0.0003345382,-0.030098192,0.022870619,-0.0017267349,-0.009055838,-0.012781693,0.07583533,0.045031916,-0.02076535,-0.07310905,-0.011597339,-0.00062336307,0.005723161,-0.018269768,0.020560576,0.023111053,-0.00881239,0.0052197427,0.022200806,0.013797317,0.019722437]::vector(768)),
  ('Smartphone', 'Latest model with high-resolution camera', 800.00, 'Electronics', 15, 'SKU12346', 'https://example.com/smartphone.jpg', '{"category" : "Electronics", "name" : "Smartphone", "description" : "Latest model with high-resolution camera"}', ARRAY[0.031757303,-0.030950155,-0.058881454,-0.05073203,0.053704526,-0.01064694,0.030361004,0.0036670829,-0.014013894,0.022840602,0.06545107,0.0108244,0.009321064,-0.0236112,0.0098358095,-0.038861487,-0.011348891,-0.011887714,0.011245335,-0.018139482,0.03049321,-0.030338986,-0.001923893,0.011787388,-0.01825618,-0.050398953,0.0036137043,-0.04487695,-0.021582587,0.023590472,-0.051335085,0.08021365,-0.06793676,-0.00514603,0.024418706,-0.054447155,-0.050472837,0.010439093,-0.017847419,0.07124281,0.004419413,-0.028902968,-0.062286377,-0.02737251,0.048311986,-0.029160773,0.0059961462,0.0344943,0.037635062,-0.081315145,0.025175434,-0.0050063017,0.023545247,-0.015210805,-0.035123624,-0.020403884,0.014771475,0.015879042,0.0029214756,0.011768866,0.004276383,-0.009031657,-0.050000243,0.059927624,-0.03906005,-0.027238877,-0.04796615,0.03084268,0.07360646,-0.028875567,0.027232852,0.015592421,0.07156161,-0.059652634,-0.04831314,-0.049740285,-0.017305655,0.10253246,0.016519215,-0.0021727297,-0.0063062175,-0.0015423468,-0.03617129,-0.03982753,-0.059866134,0.082323685,-0.01662162,-0.0048025097,0.011876321,0.08410362,-0.006159452,-0.0008565244,0.04274695,-0.08079417,0.04427687,0.04110836,0.04812812,-0.053979542,-0.004387368,-0.04829328,-0.022975856,-0.015012431,-0.0056774826,-0.03936704,0.023132714,-0.007810687,-0.011018049,0.031620245,-0.02713872,0.0018347959,-0.024968592,0.02253628,-0.00809666,-0.0076680584,0.06435103,-0.020083368,-0.0049473317,0.07430767,0.01915259,0.040656384,0.00998682,-0.014684721,0.026354978,0.032759093,0.037668057,-0.009659323,0.006720873,0.063525185,0.03982695,0.04567435,-0.02619304,-0.030550981,-0.014520635,0.0010599799,0.034034356,0.06294083,0.07422565,0.01973267,0.05249243,0.010003681,-0.034319345,-0.023254821,0.0019625498,0.033209592,-0.015176091,0.056498263,0.0041291295,-0.046049923,0.054690883,-0.021583585,-0.019928787,-0.010311507,-0.03155074,0.038876258,0.055084117,0.0006716143,-0.005959439,0.02702423,-0.0041947966,0.015374709,0.057063535,0.028639654,0.069971144,0.019529812,-0.026227735,-0.083985895,-0.0041349265,0.009833876,-0.015811538,0.016993256,-0.010458223,0.040068664,0.009195164,-0.03924835,-0.007896623,-0.06261605,0.015779363,-0.018634042,-0.0013783163,0.016493134,-0.041971806,-0.039205268,0.020863583,-0.00169911,0.026609324,-0.07237093,0.07898098,-0.008871385,0.017599586,0.018514562,-0.01763139,0.00015460308,-0.03443664,0.026305566,0.0019577034,0.049758997,-0.014016935,0.01580608,-0.005885855,-0.014773614,0.008331391,0.011858725,0.047954902,0.016360788,0.040261615,-0.014324732,0.062151354,-0.037888777,0.02075746,0.039549813,-0.077434056,0.00096539775,-0.044017132,-0.012209571,0.034755055,-0.020098051,0.008095624,0.031291816,0.04792529,-0.008659437,0.01759492,0.009537845,-0.05313831,-0.010890252,-0.03342564,0.061369378,-0.031681072,-0.053262327,8.374469e-05,-0.027414132,-0.013404388,0.033906803,0.025408141,-0.035230264,0.030235829,-0.0014981066,0.023731904,0.029274339,0.047021322,0.025153603,-0.050763946,0.042003185,0.028869675,0.023947056,-0.045773767,-0.029348088,-0.04498305,0.03974547,0.021556387,0.032411546,-0.028107764,-0.01917967,0.020117322,0.035401057,-0.087708965,0.028180089,-0.07627729,0.010020432,-0.055026818,0.013467507,0.05156387,0.030606749,-0.012557438,0.0075980667,-0.049580842,0.025251655,0.011958476,-0.05784425,-0.00688397,-0.026897762,-0.0073929257,-0.082809925,0.0707716,0.0044888635,-0.023634167,0.00959699,0.027249858,0.009045479,-0.008601681,0.007323367,0.014609572,0.007073427,0.0055342577,-0.047172364,-0.023501316,-0.03593993,-0.022744065,-0.031178312,0.007601522,0.01038201,-0.040641543,-0.02084411,-0.04739785,0.0016813428,-0.022378212,-0.024991153,-0.019224035,0.033300195,0.04363394,-0.0072962623,0.0044990415,0.00530943,-0.0061862995,-0.1226422,-0.0048183375,-0.010383665,-0.043834127,-0.010673082,0.00016926302,0.026351877,-0.03451933,-0.017912712,-0.06287377,-0.00329357,0.056648213,-0.005951308,-0.017310314,0.06057505,0.00529039,0.04522765,0.009986563,0.09290384,0.0046436884,-0.027085476,-0.0051616537,0.014926508,-0.027059292,0.07819409,0.0018491915,-0.034066174,-0.04200668,0.017987153,-0.054097146,0.0263208,-0.030290576,0.012135319,-0.053635724,0.0040904377,-0.06391213,-0.012962556,0.039401833,0.029892938,-0.010509396,-0.09667328,-0.004525119,-0.0660734,0.005074788,-0.0043580704,0.048569698,-0.029491736,-0.00813117,0.099913284,-0.02152916,-0.0046480033,-0.004279434,-0.022350302,0.07403285,2.6268553e-05,0.024700351,-0.070556544,-0.046257928,0.047623277,0.013440511,0.022684522,0.0105078975,0.029062217,0.036317576,0.012476447,-0.025555858,0.0043436335,0.006260482,-0.030046312,0.012665346,-0.060015686,-0.042867333,-0.043334395,-0.09350731,-0.015882127,-0.023036648,0.0035012013,0.019168707,-0.029792963,0.014690395,-0.03232301,0.04318316,-0.023454774,0.024906443,0.033632547,0.026205529,0.021056164,-0.014863617,0.03884084,0.019737227,0.0643725,-0.015622061,0.010209574,-0.042415053,-0.041623153,0.020822845,-0.020490937,-0.0542278,-0.0033205135,-0.041752372,-0.069488324,-0.016277319,-0.0044792043,-0.02016524,-0.03959827,-0.032634977,-0.0039365673,-0.0132405395,0.0067148125,0.075648956,-0.05606617,-0.06265819,-0.019359354,0.05813966,-0.01447109,-0.010593954,-0.00086784246,0.00957173,0.02843471,0.00845407,0.024766237,-0.017594881,-0.02089351,-0.023622723,-0.033868976,0.01189866,0.04348284,0.017560178,0.0044504236,0.0201572,-0.010445271,0.016996963,-0.063251264,0.036506347,0.014985517,-0.004923813,-0.019643096,0.004065921,-0.03441569,0.02174584,-0.022037273,-0.105745554,-0.017520802,0.024135107,-0.056571614,0.065653384,-0.11961944,-0.019004421,-0.048515763,-0.018267322,-0.02178645,-0.00048087785,-0.042244278,0.041203473,0.039137937,-0.028382456,0.0027469762,-0.035103243,-0.008536376,-0.022003518,0.013834031,0.04035347,-0.05127768,-0.021083988,-0.019288905,0.030957388,0.03837377,-0.0003459004,-0.043197013,0.059090964,0.03584024,-0.009635979,0.049144205,-0.113005035,0.012198436,0.0030250824,-0.0005766731,0.010016404,-0.004630926,0.036304604,0.030682925,-0.028248072,0.0053004674,-0.028463472,-0.045950726,-0.016214147,0.02234844,-0.024365503,0.0045087263,0.0015641076,-0.046219032,0.019860927,0.011021814,-0.024108216,-0.048900776,0.012885111,-0.0022583513,-0.030102832,-0.016490621,0.024889058,-0.0009473834,0.015075038,-0.040798195,-0.005642347,-0.0029682147,-0.050329093,0.0009567131,0.007919075,0.01719906,-0.018685095,-0.016243592,0.010302834,4.1979074e-06,-0.042400364,0.055864133,0.033395868,-0.017874744,0.0013070442,-0.05331383,-0.10789571,0.0074728676,0.03525642,-0.07436872,0.04979144,0.046753135,0.0027637088,0.014162893,-0.026069263,0.06226656,-0.056384422,0.008216318,-0.02018645,-0.007397228,0.0074180462,0.035483476,-0.01882623,-0.02706421,0.04596009,-0.013163229,-0.021003753,0.037058793,0.052453898,0.013129776,0.015402059,-0.048313417,0.023352273,0.009391176,-0.044023603,-0.0107533,0.054881006,-0.019277383,0.02055352,-0.030710667,-0.02347742,0.0092705265,-0.047558293,-0.024285497,-0.03519891,-0.0038767713,-0.005330039,-0.026968258,0.06881978,0.06537581,-0.023353418,0.01331013,0.045053896,0.032502707,0.065926,0.0009946732,0.051750924,0.005718337,-0.0038732293,-0.029579317,-0.06977859,-0.0048092776,-0.025378013,0.023722455,0.032475006,-0.031788938,-0.00917764,0.0056064464,-0.016738426,0.021969007,0.012666437,-0.046921335,-0.02513667,-0.028311022,-0.009224157,0.05264038,-0.026426777,0.02599612,-0.018745475,-0.015264339,-0.013577108,0.0011754846,0.020499794,0.01423578,-0.015937831,-0.034813095,0.06295408,-0.033208452,0.041733917,0.0022288205,-0.0036853347,-0.015074669,-0.00813031,-0.004992453,0.010502773,0.017247686,0.03162546,-0.006212466,-0.06321386,0.022924462,0.03354761,-0.02742972,-0.018287206,-0.05058406,-0.02762529,0.014693771,-0.009422438,-0.0113650765,0.04500726,-0.009418481,0.023177318,-0.0394831,0.07899207,0.010970399,0.01519068,0.060208563,-0.014248415,-0.027108915,-0.055970594,-0.05615517,0.00082430604,-0.02946103,0.012972071,-0.034580585,-0.092063755,0.023562009,0.09187191,0.03979375,-0.048233856,0.0891921,0.0054705814,-0.07132956,-0.03294508,0.015985591,-0.06979576,-0.008607954,0.03748406,0.018775256,-0.00055046624,-0.0018972756,0.010640039,-0.039262787,0.045647603,-0.052634962,-0.04485457,0.059673585,0.005487001,0.005677175,-0.040526956,-0.0023886457,-0.051557075,-0.026969707,-0.020169057,0.020184118,-0.06750348,0.014797761,0.043389246,0.022667736,0.012956063,0.056346934,0.038232267,0.02334661,-0.002965094,0.053386245,-0.016282998,-0.08433834,-0.005240998,0.020763554,0.0041468525,0.011248255,0.013354228,0.0062226793,0.01238483,-0.042322755,0.017076539,-0.024617095,-0.03331688,-0.001430632,0.05623171,0.0073584137,0.013339925,-0.0041607106,0.015201854,0.029444456,-0.039367896,0.032675862,0.016636375,0.04101005,0.0073330533,0.03937178,-0.01699229,0.026922127,-0.00465699,0.014691186,0.07985071,-0.045738634,-0.040622048,0.040370528,-0.0070402357,-0.048223954,0.048428483,-0.013764062,0.02645368,0.030109879,-0.01834218,-0.0045400057,0.036011115,-0.010352046,-0.068165384,0.037795525,-0.036501475,0.020713413,-1.2293508e-05,-0.00038850267,0.073334076,0.01821627,0.003559663,0.017506005,-0.02564981,0.039007656,-0.026543219,0.018282859,-0.038226757,-0.04996024,0.01010447,-0.012900636,-0.020180488,0.042488355,0.0135185765,-0.0083626835,-0.019743606,0.025633369,0.035687257,-0.053833067,-0.053783447,0.007418253,-0.04581871,0.032362275,0.050387084,-0.010103674,-0.051880397,0.010476682,0.015898407,0.04970622,-0.04664034,0.036457486,-0.017625386,-0.0058598807,-0.011529857,0.018154921,0.013366902,0.0021690137]::vector(768)),
  ('Coffee Maker', 'Brews coffee in under 5 minutes', 99.99, 'Kitchen Appliances', 20, 'SKU12347', 'https://example.com/coffeemaker.jpg', '{"category" : "Kitchen Appliances", "name" : "Coffee Maker", "description" : "Brews coffee in under 5 minutes"}', ARRAY[0.025002815,-0.052869678,-0.010500825,-0.024296444,0.049798742,0.043427017,-0.01307104,0.0077243242,0.022190414,0.037746448,0.029453197,-0.009484218,0.0028156517,-0.03531512,-0.012121426,0.0091221025,0.025652027,-0.009445565,-0.02820549,-0.04105274,-0.0010839493,0.015024874,0.053036522,-0.018628811,0.014746092,-0.049109433,0.026801802,-0.0070828577,-0.02369395,0.010975214,-0.03531074,0.04859645,-0.004710616,-0.018579654,-0.0076328423,-0.030808363,-0.012824788,0.03848257,0.014652247,0.058704656,0.00325119,-0.007205416,-0.04686223,-0.028575234,0.02045449,-0.008556303,-0.009746742,0.018289749,0.00093424425,-0.046003163,0.0039943205,-0.023993168,0.05866197,0.008093339,-0.00565744,-0.008198263,-0.001283407,-0.0007927462,-0.018114842,-0.008134085,-0.00014443924,0.021404255,-0.014830747,0.050932012,-0.032427747,-0.027500387,-0.020814912,0.025367612,0.061494272,-0.028271751,-0.002093295,-0.005629965,0.054627255,-0.062579386,-0.01051155,-0.06421958,-0.012094066,0.06576773,0.05998704,0.10272862,-0.021875817,-0.062225047,0.022178214,0.010618126,-0.05723891,0.040955715,-0.038523626,0.021909224,0.018677043,0.056335997,-0.01599579,0.015702266,0.025712736,-0.024550503,0.041618552,0.031751215,-0.0013378685,-0.042116627,-0.033073347,-0.011056941,0.022297822,-0.052519917,-0.06455736,0.030026494,0.04122688,0.0435459,-0.021909805,0.025392938,-0.05491582,0.022167888,-0.06104317,0.021199005,0.021531114,0.0003258208,0.051008765,-0.0056826724,0.0019850046,0.08186525,0.014742098,0.01913513,-0.026228607,-0.023587128,0.041640177,0.016765678,0.028365733,0.057187237,0.011515794,0.0734812,0.048084594,-0.0028821004,0.00025123838,-0.010272774,0.025670059,-0.049766205,0.0862307,0.07104121,0.008422137,0.026603732,0.06897059,-0.0013259795,-0.003537648,-0.016978277,-0.03289158,0.019160148,-0.030429484,0.03210423,-0.0025404708,-0.052619252,0.0020272017,-0.014941184,0.0026864705,0.012819193,-0.043763664,0.057997666,0.043563023,0.03174006,-0.04444913,0.0060016355,-0.029776296,-0.017748147,0.036185395,-0.0014833601,-0.017309692,0.04368944,0.020283954,-0.0160715,0.03019354,0.02680017,0.013467745,0.010598811,-0.009857402,0.0035379697,-0.04074403,-0.015414817,0.016311716,-0.0669727,0.0034562463,-0.024640094,-0.023524309,0.0028607736,-0.06249814,-0.058054965,-0.007223816,0.012088017,0.029124737,-0.030978883,0.07969112,-0.05076358,0.015344627,-0.00898595,-0.0097088795,-0.019155432,-0.035673082,0.027780814,0.006400352,0.055502266,-0.046420977,0.03919276,0.040964182,-0.024075434,-0.014520242,0.07941375,0.023109328,0.030869437,0.06598536,0.00059927267,0.076354064,-0.048273984,0.0025508753,0.0066330666,-0.070879966,0.03704847,-0.0650441,0.01176703,0.033744898,0.0400285,0.024317512,0.028281165,-0.008897873,-0.029537052,-0.0047060223,0.026686963,-0.07052627,0.023556747,-0.056385886,0.0714133,0.007949809,0.011887155,0.0029032454,-0.015065537,0.011513513,0.050219424,0.010533179,-0.009971522,0.03655571,-0.0066924663,-0.012303563,0.016773308,0.013691093,0.025839401,-0.044451136,0.049260832,0.05713467,0.013278825,-0.022100078,-0.0017930771,-0.016181005,0.0217466,-0.02600776,0.046996534,-0.022629611,-0.023503313,0.0074507482,0.0134722935,-0.04945182,0.022608835,-0.026130896,-0.01177188,-0.027667308,0.026118958,0.0025001818,0.021639917,-0.015105975,0.02968347,-0.043928802,0.03762012,0.019912925,-0.004347233,-0.006596504,0.016333994,-0.025137693,-0.01686705,0.04786869,0.034643404,0.011117003,-0.011134983,-0.0074818125,-0.006335571,0.022040822,-0.006491301,0.0054816976,0.038022403,0.016072717,-0.06609374,-0.03203102,-0.059326455,-0.04408214,-0.03787348,0.014894112,-0.02038928,-0.044823527,-0.015866352,-0.047105137,0.002020473,-0.04468357,0.018793538,-0.029475007,0.06967502,0.04684481,0.048074055,-0.0010090554,-0.0027273456,0.047790546,-0.030050496,0.023022242,-0.028264726,0.03571066,-0.0164874,-0.019399788,0.0076415916,-0.0060172956,-0.010469042,-0.045296766,-0.0071801674,0.032818798,0.034934863,-0.0737483,0.0327411,-0.006032433,0.05928009,-0.00927453,0.07627373,0.0050010816,-0.048511107,-0.0037969523,-0.007150538,-0.010152546,0.025746513,-0.02757783,-0.049112115,-0.029450508,0.037618097,-0.04765299,0.021502782,0.04031621,-0.021789404,-0.03477437,-0.0029428764,-0.04645585,0.015724704,0.0061205924,0.027327916,-0.016831782,-0.07413835,-0.009106179,-0.005994898,0.0015746661,-0.0066348854,0.08860898,0.026653405,-0.010490873,0.01737892,-0.036203787,0.0019658727,-0.05349199,-0.031604912,0.059320047,-0.0035595773,-0.013159466,-0.043662973,-0.000936755,0.037883844,1.1725969e-05,0.008455511,0.028007427,0.026448535,0.03587197,0.034501214,-0.020195212,-0.036874935,0.008322776,-0.038275808,0.014955824,-0.066956565,-0.03433901,-0.043297864,-0.07503335,-0.037108134,0.032691672,-0.05912909,0.023559488,-0.023238983,0.0012042256,0.0074822125,0.0058873207,-0.021845229,0.0054280413,0.05752058,-0.026954507,0.026175871,0.012301664,0.06307251,0.07353519,0.011740042,-0.012562488,-0.025707787,0.011014364,-0.064245604,-0.018075097,-0.04286179,-0.06992585,-0.031043975,-0.0022823277,-0.05855018,0.015864456,0.00024379989,0.0070141326,-0.00035948,-0.023150876,-0.063177474,0.008194795,0.023019124,0.014603101,-0.06850171,-0.07586402,-0.029384725,0.09732399,-0.023403296,0.00983274,0.00043465907,-0.037277438,0.060318034,-0.010698135,-0.0012939094,-0.015873678,-0.006272459,0.0014064384,-0.041425075,-0.021238888,0.021737115,0.030599548,0.043125883,0.01929081,-0.0011234619,-0.031159677,-0.05745639,0.0146679375,0.046521254,-0.01835481,-0.033141162,0.00036415283,-0.06466151,0.043580752,0.011921412,-0.07292401,-0.047980927,0.02159395,-0.023352068,0.0425091,-0.09635663,0.0060955146,-0.06484201,-0.029811602,-0.026076958,-0.014945281,-0.04334233,-0.00242451,0.047840517,-0.02103297,-0.0191666,-0.0074735563,1.0544848e-05,-0.028074,-0.037163526,0.030064873,-0.02934737,0.050285384,-0.023986174,0.025914317,0.10199452,-0.021887174,-0.0066847154,-0.023618985,0.03283886,0.045797225,0.047762897,-0.07030183,0.026901271,0.008702326,0.017019885,0.033345792,-0.03833666,0.031567782,-0.013102635,-0.009532979,0.025451964,-0.021708276,-0.023218581,-0.07980661,0.03028782,-0.021675726,0.03096571,-0.018742265,-0.04427001,0.009433704,0.03455316,-0.035231985,0.04002238,0.012793141,0.025124295,-0.04512409,-0.06486318,0.019942157,-0.030111039,-0.0069209165,0.0015545462,0.028818183,0.0014206765,-0.032698274,0.008883163,0.058960456,-0.00906729,0.0298577,-0.0070162034,-0.014469902,-0.0032146918,-0.04448409,0.03293327,0.040138587,0.01842061,0.0055912337,-0.03388838,-0.071546026,0.02821449,0.033089,-0.04839594,0.016159212,0.08211776,-0.08987595,0.036964364,-0.051373526,0.1035708,-0.053108595,-0.01896186,0.01644011,-0.012502358,0.008263514,0.04065409,-0.015298684,0.0011162056,0.04276282,0.0027434586,0.0324373,0.03511016,0.02446925,0.002442109,0.049384676,-0.05747281,-0.0020478321,-0.03639974,0.011938583,-0.031114291,0.03284646,-0.03238849,0.08670559,-0.07415254,-0.036738325,-0.025126172,-0.045095183,-0.015307702,-0.06554373,-0.05546525,0.005472855,-0.006981692,0.04587679,0.111925036,0.013912294,0.014268016,0.058842134,-0.011192024,0.034922387,0.012045642,0.008008024,-0.014226386,0.06913233,-0.04700873,-0.06164794,-0.0024386728,0.043209903,0.051432677,-0.017323477,0.013788927,0.012737198,0.06472892,-0.070449375,0.005222667,0.050599333,0.0015403829,0.015714316,-0.008632714,0.014941663,0.06433311,-0.021354778,-0.0071928906,-0.028242689,0.018915592,0.021451298,0.0063637616,0.0019523413,-0.017883593,0.028570741,-0.016318232,0.053636383,-0.028484613,-0.006531752,0.022900375,0.023723338,-0.024363475,-0.015181002,0.024642847,-0.002409233,-0.0001194501,0.013567875,-0.046026736,-0.016705032,-0.013025837,0.020370122,-0.027258568,-0.04735096,0.011894463,0.0019317217,-0.0031460563,0.040866848,0.00464604,0.03964947,0.027275842,-0.0030081465,-0.008669969,0.0462421,0.010375526,-0.024637504,0.08480695,-0.02768799,-0.005021901,-0.009944692,0.015040328,-0.0051919715,-0.043738216,0.054622557,-0.0116185825,-0.044851393,-0.01769878,0.06967592,-0.026938388,0.0030814619,0.07516173,-0.022243993,-0.09390373,-0.056307606,0.011178256,-0.058882743,0.016906237,0.010931337,0.011277608,-0.03310829,0.008875099,-0.017342865,-0.049926963,-0.0021014255,-0.019715691,-0.024091842,0.029629463,-0.06452303,0.009643791,-0.025999011,-0.017722748,-0.09347366,-0.019748896,-0.011190205,-0.0044534663,-0.04336357,-0.01312215,0.056558847,-0.022783643,0.0004763564,0.04152026,-0.03813543,0.0038315274,0.021157283,-0.007934057,0.0004752217,-0.057082873,-0.011285772,-0.014152046,0.03181829,0.033805694,0.04453719,-0.02024123,-0.0038247174,-0.0262423,0.007036252,-0.012817323,-0.025822328,0.06599188,0.067939,-0.022174655,-0.022773167,9.6714546e-05,-0.017627345,0.08549309,-0.06266334,-0.00575442,-0.011873023,-0.07250961,0.0056728884,0.017012162,-0.025071641,0.022021066,0.030550413,-0.010627088,0.050028834,-0.01721913,-0.050976366,-0.024867795,0.011782799,-0.075504154,-0.004392594,-0.01807583,0.031157117,0.030725744,-0.014750008,0.005684259,0.047403537,-0.08811708,0.007985649,0.043377616,-0.037903026,0.029741386,-0.0011720062,-0.010578729,0.051289707,-0.024345556,0.017949736,0.02636295,-0.059689533,0.06373776,-0.049072567,0.013506145,-0.040476285,-0.02940512,-0.023568999,-0.00035632766,0.056101788,0.061561547,0.03079068,-0.02166795,0.009211557,0.0030255727,-0.0036865661,0.023821775,0.0015869564,0.0064414316,-0.057368714,0.061502002,0.023947174,0.0046180966,-0.05202509,0.002360597,0.03557417,0.036739,-0.03005605,0.047780115,0.025282156,0.034349978,0.034781702,0.0276351,-0.040908,0.081558466]::vector(768)),
  ('Bluetooth Headphones', 'Noise cancelling, over the ear headphones', 250.00, 'Accessories', 5, 'SKU12348', 'https://example.com/headphones.jpg', '{"category" : "Accessories", "name" : "Bluetooth Headphones", "description" : "Noise cancelling, over the ear headphones"}', ARRAY[0.022783848,-0.057248034,-0.047374193,-0.04242414,0.049324054,0.0077371066,0.017048897,0.00500827,0.008471851,0.010170231,0.054357704,0.018568166,-0.024179503,0.026519066,0.026404649,-0.06330503,0.014405935,-0.015520485,0.0052459002,-0.0398403,0.0026082278,-0.026374431,0.020055598,-0.009738811,0.013321584,-0.033184614,0.034118295,-0.0011876881,-0.04513898,0.04878162,-0.0725106,0.018109042,-0.075869314,-0.023766529,0.015067321,-0.019572936,0.024169574,-0.01577634,-0.048197363,0.049358875,-0.030935159,-0.0363981,-0.04534119,-0.044748895,-0.004167742,-0.02121328,-0.052715167,0.0006209187,0.036595955,-0.085123576,0.052309636,-0.01926014,0.00049565616,-0.0057477825,0.010993081,-0.06675727,0.0037074706,-0.033420403,-0.052601676,0.023439946,-0.01880516,-0.009576131,-0.0114066675,0.10504714,0.00022831495,0.029810086,-0.0044366047,0.043377023,0.06093195,-0.004545408,0.013371212,-0.029174658,0.06625106,-0.0077476054,-0.0163617,-0.056035727,-0.024698364,0.06076837,0.020102862,0.038081013,-0.018504761,-0.027918378,0.03942784,0.004596525,-0.057653908,0.034515597,0.010063118,0.04525672,0.023651283,0.03596632,-0.0378574,-0.013078957,0.021554954,-0.0606351,-0.007272484,0.044470455,-0.015513987,-0.018171282,-0.014020262,-0.040379126,-0.032836802,-0.055859733,-0.05644243,-0.001610613,-0.05527219,-0.00052593346,-0.00546389,0.02911079,-0.0037673921,0.036246333,-0.057133533,0.043779045,-0.0028422247,-0.044305976,0.05993566,-0.005543668,-0.0015800337,0.07515586,-0.00020748413,0.03876171,0.026035579,0.012980581,0.056657698,0.020252425,0.029382393,0.011205804,0.039896134,0.04349186,0.08402962,-0.0031059172,-0.022395832,-0.023471512,0.029480197,0.0038065156,0.07106566,0.07560159,0.019708911,0.0063190344,0.06826459,0.05426478,-0.016353253,-0.016603524,0.035430502,0.01285351,-0.044608854,0.06445639,0.027575186,-0.020047447,0.07155171,-0.024042875,0.007684551,-0.057774883,-0.05863421,0.04027459,0.034241315,0.029786138,-0.011771758,-0.008067332,0.005154275,0.017256541,0.012795448,0.0361206,0.046198364,0.007581977,-0.0643159,-0.032997373,0.025989803,0.039828006,0.00950064,0.043332074,0.016609278,0.034839373,-0.022875424,-0.028605282,-0.017703732,-0.06238004,0.010994231,-0.0007306017,-0.034711856,-0.0440203,-0.025970237,-0.04595589,0.030582627,0.0073314123,-0.017986864,-0.055571377,0.082270294,-0.018736921,-0.0012149982,-0.0060279733,0.0044796504,0.025173035,-0.037219252,0.00027956237,-0.010430433,0.02825617,-0.046855696,0.018841878,0.0435598,0.005803966,0.0019149927,0.092197396,0.022937872,-0.0033373323,0.072473325,-0.014439769,0.047117453,-0.08000118,-0.012863106,0.0260884,-0.04135028,0.0070068296,-0.07510927,0.03672727,0.033531025,0.042364623,-0.019229556,0.0048453975,0.031276144,0.014006409,0.016036421,-0.017694592,-0.036794797,0.014908425,0.030831292,0.03190712,-0.022060342,0.041704472,0.017002491,-0.06408182,0.03923344,-0.02587273,-0.017719302,-0.025430005,0.06814103,-0.009046621,0.033220492,-0.033640996,-0.02523642,0.048086986,-0.035158273,0.048114188,0.043751266,0.01995209,-0.0295469,-0.020247698,-0.053099316,0.032099206,-0.045260355,0.0326798,-0.0043251985,-0.052964494,0.07017924,-0.0037189184,-0.03395965,0.040903587,-0.060891,-0.010537573,-0.030650055,-0.029651405,0.013975478,0.007255845,-0.010439494,-0.011794211,-0.05466926,0.024609366,-0.017408509,-0.05243266,-0.020957882,0.037831362,0.0216147,-0.035116594,0.03829302,-0.016048789,-0.035066966,-0.013764898,0.00042713518,0.030633073,-0.008326726,-0.015224956,0.012373721,0.0844943,0.0245434,-0.046264216,-0.011655971,-0.013199105,-0.05529712,0.006216126,0.038966317,0.04622981,-0.039118554,-0.044550307,-0.009771392,-0.006652356,-0.023040479,0.010476257,-0.004093151,0.008969803,0.010324751,-0.022387082,0.023577597,0.019100022,0.008391375,-0.07391311,-0.02210422,0.021720598,-0.0109519595,-0.0820701,0.022086475,-0.003670014,0.0019491176,-0.053155318,-0.022906458,0.0148452455,0.015515676,0.019605495,-0.02868708,-0.01828674,-0.0005499542,0.06639364,-0.01821442,0.09175476,-0.0016622626,-0.059729476,-0.019477114,0.025505545,-0.034742665,0.028956799,-0.019135797,-0.016046764,-0.03779796,0.06325585,-0.04046284,-0.0065921973,-0.0019740656,0.053527426,-0.06304376,-0.035805233,-0.04792203,-0.0012729234,0.048093352,0.007456611,-0.058022104,-0.07442454,0.012629627,-0.027595298,0.0021199721,-0.027464667,0.02698153,0.00060683774,0.044545636,0.06083593,-0.0031620082,-0.025901018,-0.034706157,0.013555886,0.042545,0.056980383,0.009854132,-0.06190446,-0.034308147,0.0043845526,0.017239122,-0.031214224,-0.010807414,0.026710719,0.022394834,-0.009421089,-0.04236166,0.022885358,0.01318956,-0.019174583,-0.0026612883,0.010784672,-0.010333064,-0.043234736,-0.054500565,-0.027753199,-0.022639737,-0.03062474,0.008183766,-0.017117208,0.03024305,-0.03615811,-0.01150264,-0.03863528,0.04852956,0.024548976,-0.012997513,-0.0041008275,0.03406041,-0.0070994645,0.072934166,0.02805505,-0.030694276,-0.035828616,-0.017640414,-0.03957751,0.06840472,0.0046152286,-0.020437988,-0.025648775,-0.083415866,-0.04167123,-0.035016168,-0.015291769,0.009293348,0.04628708,-0.014721913,-0.0033228637,0.04403616,0.061276685,0.037830554,-0.041214965,-0.084479295,-0.0012414041,0.030978376,-0.017235488,0.04445431,0.05231969,-0.0008037167,0.045372415,0.02067265,0.024952972,-0.033815585,-0.03739797,0.034983158,-0.016312862,0.017926387,-0.02016297,-0.019343764,0.017820694,-0.011671569,0.02410841,-0.042012513,-0.03900872,0.032663334,0.011938514,-0.029834026,0.047740217,-0.0058686035,-0.046729274,0.05985927,0.007610642,-0.060446266,-0.04216537,0.017497085,-0.06986214,0.076023735,-0.10476386,-0.020937927,-0.073560745,-0.014322972,-0.048601817,-0.0056885225,-0.03637434,0.04715089,0.054749545,0.014689732,0.006048463,0.046543427,-0.017363597,-0.03678888,-0.08802858,0.063708976,0.021423126,0.04030153,-0.036243204,0.036450744,0.024569608,0.016401349,-0.022465378,-0.0034262848,0.060547307,-0.014745138,-0.020591581,-0.0054737274,-0.0074623367,0.06138278,-0.016604895,0.0032445828,0.009028142,0.002864045,0.001341044,-0.03825005,0.03237135,-0.009647875,-0.0470159,-0.024240978,0.017859152,0.010279892,-0.014414872,-0.017152937,0.020384172,0.008366546,-0.003495199,-0.024638942,-0.031768326,0.06240018,-0.0067493794,-0.04322142,-0.0030645356,-0.0027114467,-0.0072583677,0.06152745,-0.05525731,0.01201016,0.034348775,-0.032004267,0.027236925,0.05926736,-0.010569189,-0.023563573,0.0018119658,0.04231199,0.01966649,-0.014960187,0.029649874,0.01606933,0.0033748902,0.021692606,0.00794783,-0.113133654,0.012736659,0.03742399,-0.010987754,0.02547777,0.026347551,-0.09020402,0.009588993,-0.043276373,0.106708415,-0.049185734,0.007007848,0.030148245,-0.026434064,-0.017702276,0.0007948643,0.026716268,0.013030543,0.013651695,-0.019745413,-0.0087912055,0.0046337135,0.038207695,0.0059329793,0.016351476,-0.069271706,-0.006662047,0.028512167,0.0038343759,-0.00027066944,0.042824432,-0.038911343,0.012483791,-0.06324616,-0.0023198558,-0.0028892683,-0.043326154,-0.035926916,-0.006348816,-0.025913015,-0.015930604,0.040185526,0.044628017,0.039083507,0.009474702,0.017115341,0.05052131,-0.01357451,0.020331299,0.038159154,0.0349774,0.015846666,0.022699736,-0.022343196,-0.056707054,-0.0010885954,0.020071063,-0.000391925,0.06397024,-0.024627347,-0.005184313,0.05034518,0.009061781,0.034097236,-0.007981921,-0.03801412,-0.0028578758,0.013567372,-0.008190868,0.033633735,-0.053976685,-0.025468381,-0.044378527,0.032747604,0.036202736,0.038062613,0.014995585,0.0036792904,-0.01603846,-0.047275733,0.066113465,0.0045884387,0.08915791,-0.0068142917,-0.0064188805,-0.060516927,-0.016080644,0.041549493,-0.008397882,-0.0071816393,0.0064753946,-0.0017467311,-0.019128935,0.0164788,0.022168875,0.011003241,-0.026863558,-0.05437178,-0.032724023,-0.0042122444,0.010392475,0.0042387135,0.04948556,-0.013747793,0.051330764,-0.0050607547,0.054571416,0.025556272,-0.00022029632,0.047628347,0.01427685,-0.020254403,-0.03590239,0.011610469,0.041846078,-0.02470694,-0.013697807,-0.021193847,-0.04341633,-0.0041078446,0.053439837,-0.021625757,-0.037942924,0.07774385,0.005912317,-0.0929516,-0.025328774,0.025199909,-0.041145753,0.017296704,-0.0050417483,0.012186051,0.0024183579,-0.025558045,-0.0051468383,-0.07548276,0.028603543,-0.04549798,0.007635448,0.010916566,-0.029269122,0.01546215,-0.024502348,-0.021702306,-0.025917016,-0.016031386,-0.0012059321,0.031981774,-0.056502126,0.025166377,0.04160211,-0.020680273,0.010293909,0.029529357,-0.01568588,0.026115898,-0.01032236,0.02089118,-0.01709118,-0.0597839,-0.029326,0.045068808,0.00761455,0.034416553,0.022160128,-0.025166225,-0.04248117,-0.029465536,0.027829373,0.006342403,-0.05032602,0.040032476,0.07705231,-0.01583979,-0.0049204687,-0.0140532,0.022657644,0.05293866,-0.009608256,-0.005003684,-0.02478457,0.029608795,-0.022698086,0.003895031,-0.0039730286,0.023749523,0.07514458,-0.036099195,0.04289709,-0.02329434,-0.06419562,0.037709154,0.0004289863,-0.02686404,0.0032855049,-0.030955583,0.018836787,0.033646755,0.022193655,-0.028475504,0.00394302,-0.05765806,-0.062945105,0.024937302,-0.06828151,0.016094193,-0.036405172,-0.016450962,0.04907368,-0.05975235,-0.04858766,0.07675482,-0.06289323,0.052024625,0.018600732,0.038572676,-0.0011811757,-0.07612922,0.03844793,-0.015206284,0.05163554,0.046980042,0.023522004,0.00037627618,0.011324654,-0.028600419,6.0430884e-05,0.00431597,-0.023766082,-0.015001608,-0.018692184,0.08730754,0.032889076,-0.018612336,-0.019428827,-0.002722986,-0.020110032,0.04016962,-0.043657966,0.044247244,0.019661218,0.042629678,0.016911589,0.038489193,-0.0036892071,0.015036206]::vector(768)),
  ('Backpack', 'Waterproof backpack with laptop compartment', 59.99, 'Accessories', 30, 'SKU12349', 'https://example.com/backpack.jpg', '{"category" : "Accessories", "name" : "Backpack", "description" : "Waterproof backpack with laptop compartment"}', ARRAY[-0.0028279827,-0.02903348,-0.02541054,-0.025740657,0.06572692,-0.01105207,-0.018005589,0.014476618,0.0039552255,0.04976717,0.034852527,0.018194634,-0.010718678,0.012003344,-0.008418802,-0.026018273,0.029329967,-0.016163627,0.009272989,-0.03639675,0.011046671,-0.008078595,0.023365447,-0.0033083789,0.020028763,-0.025491415,0.033595297,-0.0116388025,-0.057485484,0.06268812,-0.05302806,0.033510745,-0.06083909,-0.03115934,-0.014793818,-0.028653687,-0.011399838,0.03950949,-0.03437827,-0.001663737,-0.01088612,-0.01894241,-0.055767413,-0.0044360803,0.043946534,0.012161133,0.03891473,0.001239441,0.009908146,-0.07272227,0.055397917,0.003453955,0.016562339,-0.041937787,0.05197343,-0.026436094,-0.025229415,-0.034988422,-0.02628748,0.022921052,0.013600747,-0.0020118777,-0.033795673,0.06700571,0.016018055,-0.024256106,-0.02621731,0.045516666,0.05339654,0.0040287147,-0.03260985,0.0014520925,0.064204894,-0.07453437,-0.05054596,-0.042698923,-0.010596,0.013536595,0.0057951836,0.02499754,-0.008574824,-0.0074555897,-0.03567392,-0.016175417,-0.048651025,0.051804803,0.032162882,0.015001442,-0.015329716,0.028219966,-0.031235777,-0.011996138,0.001956758,-0.057833184,-0.022306677,0.031238675,-0.006414606,-0.06930158,-0.017475452,-0.027142663,0.020731354,-0.02221535,0.031049741,0.02081393,-0.022421336,0.0264318,-0.009509332,0.03522677,-0.004379289,0.011600757,-0.022017384,0.010730822,-0.010784208,-0.032706123,0.011207074,-0.023580823,0.013793131,0.05083659,0.047280807,0.048402432,0.05347524,-0.01837716,0.005956893,0.038448945,0.056967188,0.0107236095,0.03256511,0.06276655,0.04472847,0.04416061,-0.010116117,-0.048367113,0.029135885,0.010681488,0.036315914,0.056885246,0.03745567,-0.045721106,0.060501557,0.07454113,-0.018330548,-0.0113306865,-0.011580698,0.020741342,0.020118712,0.08663372,-0.009871896,0.0153012,0.05436686,-0.032210644,-0.029824084,0.023739373,-0.024163425,0.025129095,-0.016016128,0.04870382,0.013377057,0.012678613,0.011070294,-0.0072714896,0.042209458,0.029714484,0.042831365,0.032464053,-0.047759824,-0.032160178,-0.014084912,0.016434442,0.009782443,0.0013573115,-0.015243139,0.007621731,-0.037185922,-0.054615762,-0.008570435,-0.00029953485,-0.012346052,0.00016998274,-0.03163527,-0.0139267165,-0.07079747,-0.007061694,0.020720486,0.0025725542,0.019498186,-0.03700232,0.10145702,-0.004775887,-0.042089477,-0.023965659,-0.04021527,-0.0004672301,0.007410538,-0.0024715534,0.013863051,0.02261263,-0.027591249,0.020157337,0.012993745,-0.0067202765,-0.029478177,0.052134037,0.020799996,0.014809602,0.06626069,0.0069596902,0.063764,-0.04220143,-0.0040134583,0.007221788,0.014255095,0.059271786,-0.04741277,0.014235989,0.067689635,-0.005667792,0.03801926,0.0117749525,0.025480399,0.011015113,0.0037910545,0.00022392142,-0.044315543,0.010447604,0.010668871,0.0779741,-0.08010141,0.04994428,0.0024064495,-0.04755275,-0.0114773,0.014421721,-0.028229935,-0.06231835,0.05197635,-0.00798093,-0.0025467642,0.010583627,-0.017485484,0.048588324,-0.0008222008,0.033517472,0.007129084,0.0010124474,-0.05219366,0.017978905,-0.01833836,0.019664295,-0.008339645,0.013213594,8.404173e-05,-0.058585837,0.06634499,-0.032446846,-0.066239364,0.0011773852,-0.07504017,0.026009388,-0.026110237,-0.00089784985,0.004558591,-0.027107328,0.017480537,-0.0062587988,-0.008309775,0.024417007,0.022020336,-0.025295774,0.0089702625,0.026482984,0.008462929,-0.043885507,0.023143305,-0.012536918,-0.025114551,-0.030675266,-0.030063663,-0.004634334,-0.0024470752,-0.03869859,0.015594325,0.0131572345,0.0029243943,-0.046118148,-0.03834942,-0.022946607,-0.0071579637,-0.042097863,-0.01229437,0.024193348,-0.03535916,-0.05725744,-0.014191351,-0.034702625,-0.03553529,-0.0063754944,0.0024684118,0.042859882,0.013016258,-0.02985961,-0.0020391515,0.030625137,0.016144354,-0.049042817,0.024231678,-0.025589447,-0.05898161,-0.023193993,0.031626217,-0.028190944,0.017940147,-0.049932066,-0.04810013,0.047244985,0.1082508,0.001041191,-0.057233974,-0.006368648,-0.06945289,0.048442855,-0.021192377,0.10568124,0.053165488,-0.0084766345,0.031292096,-0.009400329,-0.042162478,0.06982496,-0.014560452,-0.0073914286,-0.048956916,0.030368945,0.022202695,-0.0050742053,-0.012722453,-0.011377622,-0.051865157,-0.0070718606,-0.01745792,-0.02462795,0.030636197,0.030104883,-0.04482826,-0.11079195,-0.024324121,-0.002861835,-0.014245193,-0.020608244,0.03153579,-0.009367316,0.014898636,0.033479474,-0.015162162,0.01307384,-0.052216247,-0.025208864,0.014302212,0.023454865,0.030064361,-0.00028293114,-0.05237653,0.02271106,0.0057998034,0.021696828,0.0065965196,0.061783127,0.052609395,0.018527359,-0.012383652,0.036548115,6.0759903e-05,-0.027102679,0.0020538126,-0.026467739,-0.00931995,-0.056754645,-0.059189495,0.022508893,-0.037084196,0.008752761,0.011397571,-0.001640177,0.010061019,0.024978038,0.01750796,0.0017406448,0.0692028,0.042931892,0.008515072,-0.03527143,0.006649334,-0.0015101181,0.09099013,0.0423155,-0.060909722,-0.007118597,-0.0070489836,-0.05583034,0.035233498,-0.008949495,-0.021592604,-0.023997912,-0.030185444,-0.015039309,-0.07469254,-0.05510056,0.029319923,0.01650634,-0.0660325,-0.015404232,0.03715267,0.03294396,0.005133208,-0.071616374,-0.04183193,-0.039515678,0.06556278,-0.006204309,0.018765671,0.0087025305,0.04139539,0.039423864,-0.0096283825,-0.03788884,-0.030308004,0.016888767,0.033892095,-0.0046063373,0.036512673,0.046478424,0.030432703,-0.008351917,0.038958482,0.030963391,-0.0012744869,-0.068324916,0.035514664,0.029101191,0.019952206,-0.035990257,0.05016547,-0.0034300084,0.011099454,-0.01642832,-0.055300374,-0.07178654,0.023697836,-0.02809622,0.054089297,-0.1083301,-0.018408947,-0.075191386,-0.0048826155,-0.042217527,-0.069461025,-0.06703293,0.009000863,0.06276143,-0.0017238993,0.03036515,-0.009982445,0.055421855,-0.027764114,-0.05543302,0.022685751,0.022210898,0.049183954,-0.0047965907,0.055648796,0.011152965,-0.014035957,-0.02337775,-0.01123261,0.052066986,-0.006916061,0.03199984,-0.094863154,0.003547006,0.041498255,0.004490882,0.020994756,-0.07455022,0.036187306,-0.0051827626,-0.017956927,-0.00029976605,-0.044009093,0.0028350798,-0.052361596,0.07876513,-0.06365592,0.0017824164,0.017088404,-0.038679466,-0.008001763,-0.0013830748,-0.025812596,-0.0182766,-8.765931e-05,-0.0072022257,-0.046436142,-0.072371304,0.0057044053,-0.03468649,0.056389496,-0.020051511,0.031401794,0.0026272596,-0.045338016,-0.029466175,0.008883405,0.036455907,-0.012484258,0.0015844881,0.036832172,0.023578366,-0.043958467,0.00577308,0.055652507,-0.036696434,0.002894534,-0.032786682,-0.05258521,-0.006260205,0.030400572,-0.061743345,0.021158593,0.028482735,-0.061397683,-0.015825676,0.01941984,0.075950265,-0.11372872,-0.018362995,-0.010228874,-0.009783626,0.023449693,0.027557475,-0.0023083165,-0.0021188299,0.05987247,-0.00944442,-0.020868102,0.03482851,0.039515875,-0.026193311,0.023197955,-0.07931663,0.005395495,0.013140455,-0.061495673,0.0022219154,0.038023517,-0.05545234,0.020771723,-0.0067305462,-0.03169365,-0.021337083,0.019638145,-0.053754907,-0.035756346,-0.036120877,-0.05413345,-0.0077516357,0.03129875,0.016264724,-0.011121187,0.016678393,0.0678958,-0.014889522,-0.019517552,-0.0059457496,0.018003179,-0.0072531863,0.081852585,-0.030259738,-0.05358454,0.020454926,-0.009424692,0.10091245,-0.012819172,-0.011656013,0.031110896,0.08538375,-0.026021762,0.047623295,0.04384129,-0.05093276,0.014624959,0.026958883,-0.004577614,0.02551685,-0.019736024,0.0063903728,-0.024696782,-0.041850932,0.027209712,0.0050771283,-0.028201208,-0.03125501,-0.001541728,-0.06142714,0.054404832,-0.007287412,0.0626698,0.03180891,-0.015927717,-0.04500077,-0.0022995493,0.0124429,-0.015138294,-0.026622217,0.008842311,-0.010787062,0.0010311591,0.0013770667,0.039663706,-0.02192414,-0.019322718,-0.051264115,-0.011981459,-0.03414706,-0.006800422,-0.028382706,0.043155897,-0.007300542,0.02638807,-0.019196216,0.06930381,0.020622948,0.014042502,0.06754253,-0.043790415,0.015294639,-0.040941276,0.028382495,-0.013607999,-0.040120583,0.008768077,-0.0101868035,-0.060808867,-0.013499631,0.059239235,0.035230562,-0.019976182,0.11870333,0.053272087,-0.08745547,-0.018802922,0.004555603,-0.028306624,0.0020639726,-0.018859716,0.026370116,0.0097041875,-0.0029847843,0.017317675,-0.0533067,0.038994376,-0.03322375,-0.052456018,0.050101582,-0.015041677,-0.03370439,-0.010739062,-0.039727744,-0.045931656,-0.08658831,0.05190126,0.055936754,-0.07664951,0.041408025,0.011245535,-0.012530026,0.024861438,0.016954603,0.017269976,0.06397909,-0.000105038154,0.036761504,0.006065827,-0.02139009,-0.025604198,0.010828613,0.023636553,0.04226646,0.041076142,0.025892248,-0.051934887,0.0029032188,0.040332098,-0.015436589,-0.057878137,0.005353198,0.064739525,-0.006427803,-0.024176747,0.011304507,0.03381613,0.08625095,-0.027353497,-0.039551895,-0.04934357,-0.016709028,0.024133967,0.00441431,-0.048314437,0.040782917,0.026620803,-0.02146332,0.030112874,-0.027528606,-0.016772546,0.005690125,-0.0047134855,-0.036793064,0.04092668,-0.02411072,0.023851473,0.07727627,-0.006492274,-0.0025583038,0.0017014288,-0.0541687,-0.010395329,0.031044465,-0.0536995,0.029957417,-0.040688735,-0.037072316,0.01663893,-0.04231374,-0.030213326,0.0061428403,-0.06634084,0.06036701,0.016658397,0.024410319,-0.03309207,-0.03735754,-0.04359427,-0.013476715,0.00078163255,0.033615876,0.022759296,-0.003551954,0.017715035,-0.0072518513,0.033236742,-0.0070533687,-0.05334901,-0.014660441,0.0025560227,0.03979979,-0.00433087,-0.018232862,-0.017161474,0.008870558,0.021989124,0.078787796,-0.009815632,0.022819351,0.020795409,0.028896132,-0.0061202813,0.012352534,-0.009014175,0.0024110335]::vector(768))
  ```
</details>

Here is how this table mapped to `PGVectorStore`:

- **`id_column="product_id"`**: ID column uniquely identifies each row in the products table.

- **`content_column="description"`**: The `description` column contains text descriptions of each product. This text is used by the `embedding_service` to create vectors that go in embedding_column and represent the semantic meaning of each description.

- **`embedding_column="embed"`**: The `embed` column stores the vectors created from the product descriptions. These vectors are used to find products with similar descriptions.

- **`metadata_columns=["name", "category", "price_usd", "quantity", "sku", "image_url"]`**: These columns are treated as metadata for each product. Metadata provides additional information about a product, such as its name, category, price, quantity available, SKU (Stock Keeping Unit), and an image URL. This information is useful for displaying product details in search results or for filtering and categorization.

- **`metadata_json_column="metadata"`**: The `metadata` column can store any additional information about the products in a flexible JSON format. This allows for storing varied and complex data that doesn't fit into the standard columns.


In [ ]:
# Set an existing table name
TABLE_NAME = "products"
# SCHEMA_NAME = "my_schema"

# Initialize PGVectorStore
custom_store = await PGVectorStore.create(
    engine=pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=embedding,
    # Connect to existing VectorStore by customizing below column names
    id_column="product_id",
    content_column="description",
    embedding_column="embed",
    metadata_columns=["name", "category", "price_usd", "quantity", "sku", "image_url"],
    metadata_json_column="metadata",
)

Note: 

1. Optional: If the `embed` column is newly created or has different dimensions than supported by embedding model, it is required to one-time add the embeddings for the old records, like this: 

    `ALTER TABLE products ADD COLUMN embed vector(768) DEFAULT NULL`

1. For new records, added via `VectorStore` embeddings are automatically generated.

## Hybrid Search with PGVectorStore

A Hybrid Search combines multiple lookup strategies to provide more comprehensive and relevant search results. Specifically, it leverages both dense embedding vector search (for semantic similarity) and TSV (Text Search Vector) based keyword search (for lexical matching). This approach is particularly powerful for applications requiring efficient searching through customized text and metadata, especially when a specialized embedding model isn't feasible or necessary.

By integrating both semantic and lexical capabilities, hybrid search helps overcome the limitations of each individual method:
* **Semantic Search**: Excellent for understanding the meaning of a query, even if the exact keywords aren't present. However, it can sometimes miss highly relevant documents that contain the precise keywords but have a slightly different semantic context.
* **Keyword Search**: Highly effective for finding documents with exact keyword matches and is generally fast. Its weakness lies in its inability to understand synonyms, misspellings, or conceptual relationships.

### Hybrid Search Config

You can take advantage of hybrid search with PGVectorStore using the `HybridSearchConfig`.

With a `HybridSearchConfig` provided, the `PGVectorStore` class can efficiently manage a hybrid search vector store using PostgreSQL as the backend, automatically handling the creation and population of the necessary TSV columns when possible.

### Building the config

Here are the parameters to the hybrid search config:
* **tsv_column:** The column name for TSV column. Default: `<content_column>_tsv`
* **tsv_lang:** Value representing a supported language. Default: `pg_catalog.english`
* **fts_query:** If provided, this would be used for secondary retrieval instead of user provided query.
* **fusion_function:** Determines how the results are to be merged, default is equal weighted sum ranking.
* **fusion_function_parameters:** Parameters for the fusion function
* **primary_top_k:** Max results fetched for primary retrieval. Default: `4`
* **secondary_top_k:** Max results fetched for secondary retrieval. Default: `4`
* **index_name:** Name of the index built on the `tsv_column`
* **index_type:** GIN or GIST. Default: `GIN`

Here is an example `HybridSearchConfig`

In [ ]:
from langchain_postgres.v2.hybrid_search_config import (
    HybridSearchConfig,
    reciprocal_rank_fusion,
)

hybrid_search_config = HybridSearchConfig(
    tsv_column="hybrid_description",
    tsv_lang="pg_catalog.english",
    fusion_function=reciprocal_rank_fusion,
    fusion_function_parameters={
        "rrf_k": 60,
        "fetch_top_k": 10,
    },
)

**Note:** In this case, we have mentioned the fusion function to be a `reciprocal rank fusion` but you can also use the `weighted_sum_ranking`.

Make sure to use the right fusion function parameters

`reciprocal_rank_fusion`:
* rrf_k: The RRF parameter k. Defaults to 60
* fetch_top_k: The number of documents to fetch after merging the results. Defaults to 4

`weighted_sum_ranking`:
* primary_results_weight: The weight for the primary source's scores. Defaults to 0.5
* secondary_results_weight: The weight for the secondary source's scores. Defaults to 0.5
* fetch_top_k: The number of documents to fetch after merging the results. Defaults to 4


### Usage

Let's assume we are using the previously mentioned table [`products`](#create-a-vector-store-using-existing-table), which stores product details for an eComm venture.


#### With a new hybrid search table
To create a new postgres table with the tsv column, specify the hybrid search config during the initialization of the vector store.

In this case, all the similarity searches will make use of hybrid search.

In [ ]:
from langchain_postgres import PGVectorStore

TABLE_NAME = "hybrid_search_products"

await pg_engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    vector_size=VECTOR_SIZE,
    id_column="product_id",
    content_column="description",
    embedding_column="embed",
    metadata_columns=["name", "category", "price_usd", "quantity", "sku", "image_url"],
    metadata_json_column="metadata",
    hybrid_search_config=hybrid_search_config,
    store_metadata=True,
)

vs_hybrid = await PGVectorStore.create(
    pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=embedding,
    # Connect to existing VectorStore by customizing below column names
    id_column="product_id",
    content_column="description",
    embedding_column="embed",
    metadata_columns=["name", "category", "price_usd", "quantity", "sku", "image_url"],
    metadata_json_column="metadata",
    hybrid_search_config=hybrid_search_config,
)

# Fetch documents from the previously created store to fetch product documents
docs = await custom_store.asimilarity_search("products", k=5)
# Add data normally to the hybrid search vector store, which will also add the tsv values in tsv_column
await vs_hybrid.aadd_documents(docs)

# Use hybrid search
hybrid_docs = await vs_hybrid.asimilarity_search("products", k=5)
print(hybrid_docs)

#### With a pre-existing table

If a hybrid search config is **NOT** provided during `init_vectorstore_table` while creating a table, the table will not contain a tsv_column. In this case you can still take advantage of hybrid search using the `HybridSearchConfig`.

The specified TSV column is not present but the TSV vectors are created dynamically on-the-go for hybrid search.

In [ ]:
from langchain_postgres import PGVectorStore

# Set the existing table name
TABLE_NAME = "products"
# SCHEMA_NAME = "my_schema"

hybrid_search_config = HybridSearchConfig(
    tsv_lang="pg_catalog.english",
    fusion_function=reciprocal_rank_fusion,
    fusion_function_parameters={
        "rrf_k": 60,
        "fetch_top_k": 10,
    },
)

# Initialize PGVectorStore with the hybrid search config
custom_hybrid_store = await PGVectorStore.create(
    pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=embedding,
    # Connect to existing VectorStore by customizing below column names
    id_column="product_id",
    content_column="description",
    embedding_column="embed",
    metadata_columns=["name", "category", "price_usd", "quantity", "sku", "image_url"],
    metadata_json_column="metadata",
    hybrid_search_config=hybrid_search_config,
)

# Use hybrid search
hybrid_docs = await custom_hybrid_store.asimilarity_search("products", k=5)
print(hybrid_docs)

In this case, all the similarity searches will make use of hybrid search.

#### Applying Hybrid Search to Specific Queries

To use hybrid search only for certain queries, omit the configuration during initialization and pass it directly to the search method when needed.


In [ ]:
# Use hybrid search
hybrid_docs = await custom_store.asimilarity_search(
    "products", k=5, hybrid_search_config=hybrid_search_config
)
print(hybrid_docs)

### Hybrid Search Index

Optionally, if you have created a Postgres table with a tsv_column, you can create an index.

In [ ]:
await vs_hybrid.aapply_hybrid_search_index()

## Clean up

**⚠️ WARNING: this can not be undone**

Drop the vector store table.

In [ ]:
await pg_engine.adrop_table(TABLE_NAME)

## Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials](/docs/tutorials/rag)
- [How-to: Question and answer with RAG](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](https://python.langchain.com/docs/concepts/retrieval)

## API reference

For detailed documentation of all VectorStore features and configurations head to the API reference: https://python.langchain.com/api_reference/postgres/v2/langchain_postgres.v2.vectorstores.PGVectorStore.html